# Assignment

here comes info about the assignment

## Import libraries

here come all the imported libraries

In [10]:
import pandas as pd
from pathlib import Path
import numpy as np
import math
import scipy
from scipy.signal import find_peaks
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd


## Part I - Data import

Here comes the first part of the assignment. Importing the filtered data. Create a dataframe.
Show the data in a table.


In [11]:
#file_path = 'Greenhouse_gas_emissions.csv'
file_path = 'NOx_data.csv'
#file_path = '2.5.csv'
#file_path = '10.csv'

df = pd.read_csv(file_path, sep = ';') 
# The file had an incorrect delimiter which caused a ParserError,
# this is solved by adding sep=';' when reading the file. 

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfc in position 767: invalid start byte

In [ ]:
df

,TIME,2020,Unnamed: 2,2019
0,Belgium,-480.47,NaN,-334.9
1,Bulgaria,"-9,365.55",NaN,"-9,405.54"
2,Czechia,"7,778.07",NaN,"11,267.85"
3,Denmark,"2,953.37",NaN,"3,101.71"
4,Germany,"-6,822.26",NaN,"4,196.84"
5,Estonia,"1,149.49",NaN,"2,509.07"
6,Ireland,"6,657.07",NaN,"7,042.45"
7,Greece,"-5,400.73",NaN,"-5,416.18"
8,Spain,"-45,229.39",NaN,"-44,094.76"
9,France,"-18,226.97",NaN,"-21,621.85"


In [ ]:
# Maybe we should merge the data so that we have one row per country (instead of now per city within a country)
# We need to filter the data, get rid of columns that we will not look at.

## Part II - Data processing

Peak finding. Check of common peaks between data sets. 

## Part III - Data visualisation

Conclusion. Relate found info from Part II to our RQ. Perhaps with graphs, maps, etc. 